In [1]:
import chess
import chess.pgn
import pandas as pd
import psycopg2 
import io
from config import password
from sqlalchemy import create_engine
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
rds_connection_string = f"postgres:{password}@localhost:5432/chessDBv2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
frenchDefenseTable = pd.read_sql_query("SELECT * FROM scrub_table WHERE openingvariation LIKE 'French Defense%%'", con=engine)
frenchDefenseTable["openingvariation"] = "French Defense"
frenchDefenseTable = frenchDefenseTable.drop(columns=['whiteplayername', 'blackplayername',
                                                      'whiteelo', 'blackelo', 'gamepgn'])

In [4]:
frenchDefenseTable.head(3)

,openingvariation,gamefen
0,French Defense,rnbqk1nr/pp1p1ppp/2p1p3/8/2PPP3/8/PP1b1PPP/RN1...
1,French Defense,rnbqk2r/ppp1bppp/3ppn2/8/3PP3/2N2N2/PPP2PPP/R1...
2,French Defense,rnbqkbnr/pp3ppp/8/2pp4/3P4/2N5/PPP2PPP/R1BQKBNR


In [5]:
queensGambitTable = pd.read_sql_query("SELECT * FROM scrub_table WHERE openingvariation LIKE 'Queen_s Gambit%%'", con=engine)
queensGambitTable["openingvariation"] = "Queens Gambit"
queensGambitTable = queensGambitTable.drop(columns=['whiteplayername', 'blackplayername',
                                                      'whiteelo', 'blackelo', 'gamepgn'])

In [6]:
queensGambitTable.head(3)

,openingvariation,gamefen
0,Queens Gambit,r1bqkb1r/ppp1pppp/2n2n2/8/2pP1B2/2N5/PP2PPPP/R...
1,Queens Gambit,rnbqkb1r/pp3ppp/2p1pn2/3p2B1/2PP4/2N5/PP2PPPP/...
2,Queens Gambit,rnbqk2r/ppp1bppp/4pn2/2Pp4/3P4/2N5/PP2PPPP/R1B...


In [7]:
testingTable = frenchDefenseTable.append(queensGambitTable)

In [8]:
sicilianDefenseTable = pd.read_sql_query("SELECT * FROM scrub_table WHERE openingvariation LIKE 'Sicilian Defense%%'", con=engine)
sicilianDefenseTable["openingvariation"] = "Sicilian Defense"
sicilianDefenseTable = sicilianDefenseTable.drop(columns=['whiteplayername', 'blackplayername',
                                                      'whiteelo', 'blackelo', 'gamepgn'])

In [9]:
sicilianDefenseTable.head(3)

,openingvariation,gamefen
0,Sicilian Defense,rnbqkb1r/pp2pppp/3p1n2/8/3NP3/8/PPP2PPP/RNBQKB1R
1,Sicilian Defense,r1bqk1nr/pp1pppbp/2n3p1/2p5/2B1P3/2NP4/PPP2PPP...
2,Sicilian Defense,rnbqkbnr/1p3ppp/p2pp3/2p5/2B1P3/2N2N2/PPPP1PPP...


In [10]:
testingTable = testingTable.append(sicilianDefenseTable)

In [11]:
dutchDefenseTable = pd.read_sql_query("SELECT * FROM scrub_table WHERE openingvariation LIKE 'Dutch%%'", con=engine)
dutchDefenseTable["openingvariation"] = "Dutch Defense"
dutchDefenseTable = dutchDefenseTable.drop(columns=['whiteplayername', 'blackplayername',
                                                      'whiteelo', 'blackelo', 'gamepgn'])

In [12]:
dutchDefenseTable.head(3)

,openingvariation,gamefen
0,Dutch Defense,rnbqk2r/ppppp1bp/5np1/5p2/3P1P2/4PN2/PPP3PP/RN...
1,Dutch Defense,rnbqkbnr/ppp1p3/3p3p/5pp1/2PP4/4PN2/PP3PPP/RNB...
2,Dutch Defense,rnbqk2r/ppppp1bp/5np1/5p2/2PP4/2N2N2/PP2PPPP/R...


In [13]:
testingTable = testingTable.append(dutchDefenseTable)

In [14]:
def boardToArray(boardPosition):
    
    boardArray = []
    rowList = boardPosition.split("/")

    for i in range(len(rowList)):
        rowArray = []
        for j in rowList[i]:
            try:
                rowArray += int(j) * [0]
            except:
                if(j=='r'):
                    rowArray.append(1)
                elif(j=='n'):
                    rowArray.append(2)
                elif(j=='b'):
                    rowArray.append(3)
                elif(j=='q'):
                    rowArray.append(4)
                elif(j=='k'):
                    rowArray.append(5)
                elif(j=='p'):
                    rowArray.append(6)
                elif(j=='R'):
                    rowArray.append(7)
                elif(j=='N'):
                    rowArray.append(8)
                elif(j=='B'):
                    rowArray.append(9)
                elif(j=='Q'):
                    rowArray.append(10)
                elif(j=='K'):
                    rowArray.append(11)
                elif(j=='P'):
                    rowArray.append(12)
        boardArray.append(rowArray)
        
    return boardArray

In [15]:
gameFenList = testingTable["gamefen"].to_list()

In [16]:
boardArrayList = []
for i in range(len(gameFenList)):
    npArray = np.array(boardToArray(gameFenList[i]))
    npArray = npArray.flatten()
    boardArrayList.append(npArray)

In [17]:
testingTable['boardstate'] = boardArrayList

In [18]:
testingTable = testingTable.drop(columns=['gamefen'])

In [19]:
testingTable['openingvariation'].iloc[0]

'French Defense'

In [20]:
for i in range(len(testingTable)):
    if(testingTable['openingvariation'].iloc[i] == 'Queens Gambit'):
        testingTable['openingvariation'].iloc[i] = 0
    elif(testingTable['openingvariation'].iloc[i] == 'French Defense'):
        testingTable['openingvariation'].iloc[i] = 1
    elif(testingTable['openingvariation'].iloc[i] == 'Sicilian Defense'):
        testingTable['openingvariation'].iloc[i] = 2
    elif(testingTable['openingvariation'].iloc[i] == 'Dutch Defense'):
        testingTable['openingvariation'].iloc[i] = 3

In [21]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist

In [22]:
print(len(testingTable['boardstate']))

36909


In [23]:
testingTable['boardstate'].iloc[0]

array([ 1,  2,  3,  4,  5,  0,  2,  1,  6,  6,  0,  6,  0,  6,  6,  6,  0,
        0,  6,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       12, 12, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12,  0,
        3,  0, 12, 12, 12,  7,  8,  0, 10, 11,  9,  8,  7])

In [24]:
squareArray = [[] for x in range(64)]

for i in range(len(testingTable['boardstate'])): #(line i)
    print(f'game{i}')
    for j in range(len(testingTable['boardstate'].iloc[i])): #(each element in line i)
        #print('appending ' + str(testingTable["boardstate"].iloc[i][j]) + f' to list {j}')
        squareArray[j].append(testingTable['boardstate'].iloc[i][j])

game0
game1
game2
game3
game4
game5
game6
game7
game8
game9
game10
game11
game12
game13
game14
game15
game16
game17
game18
game19
game20
game21
game22
game23
game24
game25
game26
game27
game28
game29
game30
game31
game32
game33
game34
game35
game36
game37
game38
game39
game40
game41
game42
game43
game44
game45
game46
game47
game48
game49
game50
game51
game52
game53
game54
game55
game56
game57
game58
game59
game60
game61
game62
game63
game64
game65
game66
game67
game68
game69
game70
game71
game72
game73
game74
game75
game76
game77
game78
game79
game80
game81
game82
game83
game84
game85
game86
game87
game88
game89
game90
game91
game92
game93
game94
game95
game96
game97
game98
game99
game100
game101
game102
game103
game104
game105
game106
game107
game108
game109
game110
game111
game112
game113
game114
game115
game116
game117
game118
game119
game120
game121
game122
game123
game124
game125
game126
game127
game128
game129
game130
game131
game132
game133
game134
game135
game136
game137
game13

game1051
game1052
game1053
game1054
game1055
game1056
game1057
game1058
game1059
game1060
game1061
game1062
game1063
game1064
game1065
game1066
game1067
game1068
game1069
game1070
game1071
game1072
game1073
game1074
game1075
game1076
game1077
game1078
game1079
game1080
game1081
game1082
game1083
game1084
game1085
game1086
game1087
game1088
game1089
game1090
game1091
game1092
game1093
game1094
game1095
game1096
game1097
game1098
game1099
game1100
game1101
game1102
game1103
game1104
game1105
game1106
game1107
game1108
game1109
game1110
game1111
game1112
game1113
game1114
game1115
game1116
game1117
game1118
game1119
game1120
game1121
game1122
game1123
game1124
game1125
game1126
game1127
game1128
game1129
game1130
game1131
game1132
game1133
game1134
game1135
game1136
game1137
game1138
game1139
game1140
game1141
game1142
game1143
game1144
game1145
game1146
game1147
game1148
game1149
game1150
game1151
game1152
game1153
game1154
game1155
game1156
game1157
game1158
game1159
game1160
game1161
g

game2273
game2274
game2275
game2276
game2277
game2278
game2279
game2280
game2281
game2282
game2283
game2284
game2285
game2286
game2287
game2288
game2289
game2290
game2291
game2292
game2293
game2294
game2295
game2296
game2297
game2298
game2299
game2300
game2301
game2302
game2303
game2304
game2305
game2306
game2307
game2308
game2309
game2310
game2311
game2312
game2313
game2314
game2315
game2316
game2317
game2318
game2319
game2320
game2321
game2322
game2323
game2324
game2325
game2326
game2327
game2328
game2329
game2330
game2331
game2332
game2333
game2334
game2335
game2336
game2337
game2338
game2339
game2340
game2341
game2342
game2343
game2344
game2345
game2346
game2347
game2348
game2349
game2350
game2351
game2352
game2353
game2354
game2355
game2356
game2357
game2358
game2359
game2360
game2361
game2362
game2363
game2364
game2365
game2366
game2367
game2368
game2369
game2370
game2371
game2372
game2373
game2374
game2375
game2376
game2377
game2378
game2379
game2380
game2381
game2382
game2383
g

game3349
game3350
game3351
game3352
game3353
game3354
game3355
game3356
game3357
game3358
game3359
game3360
game3361
game3362
game3363
game3364
game3365
game3366
game3367
game3368
game3369
game3370
game3371
game3372
game3373
game3374
game3375
game3376
game3377
game3378
game3379
game3380
game3381
game3382
game3383
game3384
game3385
game3386
game3387
game3388
game3389
game3390
game3391
game3392
game3393
game3394
game3395
game3396
game3397
game3398
game3399
game3400
game3401
game3402
game3403
game3404
game3405
game3406
game3407
game3408
game3409
game3410
game3411
game3412
game3413
game3414
game3415
game3416
game3417
game3418
game3419
game3420
game3421
game3422
game3423
game3424
game3425
game3426
game3427
game3428
game3429
game3430
game3431
game3432
game3433
game3434
game3435
game3436
game3437
game3438
game3439
game3440
game3441
game3442
game3443
game3444
game3445
game3446
game3447
game3448
game3449
game3450
game3451
game3452
game3453
game3454
game3455
game3456
game3457
game3458
game3459
g

game4434
game4435
game4436
game4437
game4438
game4439
game4440
game4441
game4442
game4443
game4444
game4445
game4446
game4447
game4448
game4449
game4450
game4451
game4452
game4453
game4454
game4455
game4456
game4457
game4458
game4459
game4460
game4461
game4462
game4463
game4464
game4465
game4466
game4467
game4468
game4469
game4470
game4471
game4472
game4473
game4474
game4475
game4476
game4477
game4478
game4479
game4480
game4481
game4482
game4483
game4484
game4485
game4486
game4487
game4488
game4489
game4490
game4491
game4492
game4493
game4494
game4495
game4496
game4497
game4498
game4499
game4500
game4501
game4502
game4503
game4504
game4505
game4506
game4507
game4508
game4509
game4510
game4511
game4512
game4513
game4514
game4515
game4516
game4517
game4518
game4519
game4520
game4521
game4522
game4523
game4524
game4525
game4526
game4527
game4528
game4529
game4530
game4531
game4532
game4533
game4534
game4535
game4536
game4537
game4538
game4539
game4540
game4541
game4542
game4543
game4544
g

...

game33296
game33297
game33298
game33299
game33300
game33301
game33302
game33303
game33304
game33305
game33306
game33307
game33308
game33309
game33310
game33311
game33312
game33313
game33314
game33315
game33316
game33317
game33318
game33319
game33320
game33321
game33322
game33323
game33324
game33325
game33326
game33327
game33328
game33329
game33330
game33331
game33332
game33333
game33334
game33335
game33336
game33337
game33338
game33339
game33340
game33341
game33342
game33343
game33344
game33345
game33346
game33347
game33348
game33349
game33350
game33351
game33352
game33353
game33354
game33355
game33356
game33357
game33358
game33359
game33360
game33361
game33362
game33363
game33364
game33365
game33366
game33367
game33368
game33369
game33370
game33371
game33372
game33373
game33374
game33375
game33376
game33377
game33378
game33379
game33380
game33381
game33382
game33383
game33384
game33385
game33386
game33387
game33388
game33389
game33390
game33391
game33392
game33393
game33394
game33395


game34152
game34153
game34154
game34155
game34156
game34157
game34158
game34159
game34160
game34161
game34162
game34163
game34164
game34165
game34166
game34167
game34168
game34169
game34170
game34171
game34172
game34173
game34174
game34175
game34176
game34177
game34178
game34179
game34180
game34181
game34182
game34183
game34184
game34185
game34186
game34187
game34188
game34189
game34190
game34191
game34192
game34193
game34194
game34195
game34196
game34197
game34198
game34199
game34200
game34201
game34202
game34203
game34204
game34205
game34206
game34207
game34208
game34209
game34210
game34211
game34212
game34213
game34214
game34215
game34216
game34217
game34218
game34219
game34220
game34221
game34222
game34223
game34224
game34225
game34226
game34227
game34228
game34229
game34230
game34231
game34232
game34233
game34234
game34235
game34236
game34237
game34238
game34239
game34240
game34241
game34242
game34243
game34244
game34245
game34246
game34247
game34248
game34249
game34250
game34251


game35018
game35019
game35020
game35021
game35022
game35023
game35024
game35025
game35026
game35027
game35028
game35029
game35030
game35031
game35032
game35033
game35034
game35035
game35036
game35037
game35038
game35039
game35040
game35041
game35042
game35043
game35044
game35045
game35046
game35047
game35048
game35049
game35050
game35051
game35052
game35053
game35054
game35055
game35056
game35057
game35058
game35059
game35060
game35061
game35062
game35063
game35064
game35065
game35066
game35067
game35068
game35069
game35070
game35071
game35072
game35073
game35074
game35075
game35076
game35077
game35078
game35079
game35080
game35081
game35082
game35083
game35084
game35085
game35086
game35087
game35088
game35089
game35090
game35091
game35092
game35093
game35094
game35095
game35096
game35097
game35098
game35099
game35100
game35101
game35102
game35103
game35104
game35105
game35106
game35107
game35108
game35109
game35110
game35111
game35112
game35113
game35114
game35115
game35116
game35117


game35910
game35911
game35912
game35913
game35914
game35915
game35916
game35917
game35918
game35919
game35920
game35921
game35922
game35923
game35924
game35925
game35926
game35927
game35928
game35929
game35930
game35931
game35932
game35933
game35934
game35935
game35936
game35937
game35938
game35939
game35940
game35941
game35942
game35943
game35944
game35945
game35946
game35947
game35948
game35949
game35950
game35951
game35952
game35953
game35954
game35955
game35956
game35957
game35958
game35959
game35960
game35961
game35962
game35963
game35964
game35965
game35966
game35967
game35968
game35969
game35970
game35971
game35972
game35973
game35974
game35975
game35976
game35977
game35978
game35979
game35980
game35981
game35982
game35983
game35984
game35985
game35986
game35987
game35988
game35989
game35990
game35991
game35992
game35993
game35994
game35995
game35996
game35997
game35998
game35999
game36000
game36001
game36002
game36003
game36004
game36005
game36006
game36007
game36008
game36009


game36825
game36826
game36827
game36828
game36829
game36830
game36831
game36832
game36833
game36834
game36835
game36836
game36837
game36838
game36839
game36840
game36841
game36842
game36843
game36844
game36845
game36846
game36847
game36848
game36849
game36850
game36851
game36852
game36853
game36854
game36855
game36856
game36857
game36858
game36859
game36860
game36861
game36862
game36863
game36864
game36865
game36866
game36867
game36868
game36869
game36870
game36871
game36872
game36873
game36874
game36875
game36876
game36877
game36878
game36879
game36880
game36881
game36882
game36883
game36884
game36885
game36886
game36887
game36888
game36889
game36890
game36891
game36892
game36893
game36894
game36895
game36896
game36897
game36898
game36899
game36900
game36901
game36902
game36903
game36904
game36905
game36906
game36907
game36908


In [25]:
squaresDf = pd.DataFrame.from_records(squareArray)
squaresDf.head()

,0,1,2,3,4,5,6,7,8,9,...,36899,36900,36901,36902,36903,36904,36905,36906,36907,36908
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2,2,2,2,2,0,2,2,2,2,...,2,2,2,2,2,0,2,2,2,2
2,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,4,4,4,0,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
4,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [26]:
squaresDf = squaresDf.transpose()


In [27]:
squaresDf.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1,2,3,4,5,0,2,1,6,6,...,12,12,7,8,0,10,11,9,8,7
1,1,2,3,4,5,0,0,1,6,6,...,12,12,7,0,9,10,11,9,0,7
2,1,2,3,4,5,3,2,1,6,6,...,12,12,7,0,9,10,11,9,8,7
3,1,2,3,0,5,3,2,1,6,6,...,12,12,7,8,9,4,11,9,0,7
4,1,2,3,4,5,0,2,1,6,6,...,12,12,7,8,9,10,11,9,0,7


In [28]:
X_train, X_test, y_train, y_test = train_test_split(squaresDf, testingTable['openingvariation'], test_size=0.1, random_state=42)


In [29]:
print(y_train.shape)


(33218,)


In [30]:
X_train.shape

(33218, 64)

In [31]:
num_classes = 4
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
model = Sequential()

In [32]:
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))

In [33]:
model.add(Dense(100, activation='relu'))

In [34]:
model.add(Dense(num_classes, activation='softmax'))

In [35]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               6500      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 17,004
Trainable params: 17,004
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [38]:
model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2
)

Train on 33218 samples
Epoch 1/10
33218/33218 - 1s - loss: 0.1558 - accuracy: 0.9463
Epoch 2/10
33218/33218 - 1s - loss: 0.0890 - accuracy: 0.9688
Epoch 3/10
33218/33218 - 1s - loss: 0.0789 - accuracy: 0.9726
Epoch 4/10
33218/33218 - 1s - loss: 0.0713 - accuracy: 0.9739
Epoch 5/10
33218/33218 - 1s - loss: 0.0687 - accuracy: 0.9754
Epoch 6/10
33218/33218 - 1s - loss: 0.0653 - accuracy: 0.9762
Epoch 7/10
33218/33218 - 1s - loss: 0.0631 - accuracy: 0.9767
Epoch 8/10
33218/33218 - 1s - loss: 0.0610 - accuracy: 0.9774
Epoch 9/10
33218/33218 - 1s - loss: 0.0598 - accuracy: 0.9776
Epoch 10/10
33218/33218 - 1s - loss: 0.0574 - accuracy: 0.9787


In [39]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3691/3691 - 0s - loss: 0.0557 - accuracy: 0.9791
Loss: 0.05566895269392403, Accuracy: 0.9791384339332581


In [40]:
test = np.expand_dims(X_train.iloc[32525], axis=0)
print(test.shape)
print(test)

(1, 64)
[[ 1  2  3  0  5  3  2  1  6  6  6  0  0  6  6  6  0  0  0  4  6  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8  0  0  8  0  0
  12 12 12 12  0 12 12 12  7  0  9 10 11  9  0  7]]


In [41]:
y_train

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [42]:
print(f"One-Hot-Encoded Prediction: {model.predict(test).round()}")
print(f"Predicted class: {model.predict_classes(test)}")

One-Hot-Encoded Prediction: [[0. 1. 0. 0.]]
Predicted class: [1]
